In [11]:
# get ratios for segment analysis 
# use BL of whole campaign 
# date created: 3/2/2020
# author: sofia chelpon 


In [12]:
import numpy as np 
import xarray as xr
import pandas as pd 
import datetime 
import seaborn as sns

import datetime as dt 

import matplotlib
from matplotlib import pyplot as plt
import matplotlib.ticker as mticker
from mpl_toolkits import mplot3d

%matplotlib inline
%config inlinebackend.figure_format='retina'

In [17]:
#### ------------------- LOAD IN 
# data overall 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
toga_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/toga_data_df.pkl")

# segment data
awas_segments = pd.read_pickle("./awas_segments.pkl")
awas_segments = awas_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON').drop(columns = 'Notes')
# segment data 
toga_segments = pd.read_pickle("./toga_segments.pkl")
toga_segments = toga_segments.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns ='GGLON')

,Time_UTC,Flight,Intrument,C2H6_Ethane,C2H2_Ethyne,C3H8_Propane,C4H10_Isobutane,C4H10_nButane,C5H12_nPentane,C6H6_Benzene,...,C2Br2F4_Halon_2402,CH2BrCl_Bromochloromethane,CH2Br2_Methylene_bromide,CHBrCl2_Bromodichloromethane,CHBr2Cl_Dibromochloromethane,CHBr3_Bromoform,C2H5ONO2_Ethyl_nitrate,C3H7NO3_Isopropyl_nitrate,C4H9NO3_2_Butyl_nitrate,C4H9NO3_n_Butyl_nitrate
13,2014-01-22 01:44:00,RF05,AWAS,418.0,38.0,22.0,1.1,1.500000,NaN,4.6,...,0.43,0.14,0.99,0.48,0.25,0.74,3.23,1.62,0.67,0.00
14,2014-01-22 01:55:15,RF05,AWAS,422.0,42.0,25.0,1.4,2.200000,NaN,4.9,...,0.41,0.13,0.98,NaN,0.23,0.70,3.25,1.52,0.78,0.00
15,2014-01-22 02:06:30,RF05,AWAS,360.0,34.0,15.0,0.9,2.500000,0.2,3.6,...,0.41,0.12,0.96,NaN,0.21,0.67,3.07,1.38,0.54,0.00
16,2014-01-22 02:17:45,RF05,AWAS,344.0,34.0,15.0,0.9,1.900000,NaN,3.5,...,0.40,0.15,0.99,NaN,0.24,0.80,3.79,1.38,0.65,0.00
17,2014-01-22 02:29:00,RF05,AWAS,556.0,75.0,88.0,10.3,16.299999,2.1,15.7,...,0.41,0.17,1.01,0.42,0.25,0.81,4.13,2.66,0.99,1.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512,2014-02-22 01:15:43,RF14,AWAS,495.0,47.0,31.0,1.6,2.100000,0.2,7.1,...,0.40,0.13,0.83,0.44,0.16,0.48,1.69,0.76,0.15,0.00
513,2014-02-22 01:24:44,RF14,AWAS,482.0,50.0,30.0,1.3,1.600000,NaN,6.5,...,0.42,0.16,0.78,NaN,0.09,0.33,1.64,0.36,0.11,0.00
514,2014-02-22 01:33:43,RF14,AWAS,481.0,51.0,29.0,1.6,2.100000,NaN,7.7,...,0.41,0.17,0.80,0.38,0.13,0.44,1.69,0.78,0.14,0.00
515,2014-02-22 01:42:44,RF14,AWAS,460.0,47.0,23.0,0.9,2.100000,NaN,6.9,...,0.42,0.12,0.76,0.23,0.12,0.40,1.67,0.58,0.13,0.00


In [20]:
# save positions of each segment 
awas_df = pd.read_pickle("/Volumes/scdrive2/TTS_2020/get_ratios/awas_data_df.pkl")
awas_df_ut = awas_df.loc[(awas_df['GGALT'] < 14000) & (awas_df['GGALT'] > 12000)]

awas_positions = awas_df_ut.transpose()
awas_positions = awas_positions[:7]
awas_positions.to_pickle("./awas_positions.pkl")

In [5]:
##### --------------------- get campaign average BL means 
# AWAS 
awas_bl = pd.Series()
awas_bl = awas_df.loc[(awas_df['GGALT'] < 2000)].mean()
awas_bl = awas_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
awas_bl.index.name = "Trace_Gas"

# TOGA
toga_bl = pd.Series()
toga_bl = toga_df.loc[(toga_df['GGALT'] < 2000)].mean()
toga_bl = toga_bl.drop('GGALT').drop('GGLAT').drop('GGLON')
toga_bl.index.name = "Trace_Gas"

In [6]:
######## ------------- GET AWAS RATIOS 
# thing to fill with ratios 
awas_seg_ratios = pd.DataFrame()
for col in awas_df.columns: 
    awas_seg_ratios[col] = ''
awas_seg_ratios = awas_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(awas_segments))):
    # find segment, get info 
    awas_ut = awas_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': awas_ut['Time_UTC'], 'Flight': awas_ut['Flight']})
    # get ratio 
    awas_ut = awas_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = awas_ut/awas_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    awas_seg_ratios = awas_seg_ratios.append(to_append, ignore_index = True)

In [7]:
######## ------------- GET TOGA RATIOS 
# thing to fill with ratios 
toga_seg_ratios = pd.DataFrame()
for col in toga_df.columns: 
    toga_seg_ratios[col] = ''
toga_seg_ratios = toga_seg_ratios.drop(columns = 'GGALT').drop(columns = 'GGLAT').drop(columns = 'GGLON').drop(columns = 'Notes').drop(columns = 'Intrument')
    
# loop through every segment and get ratio for it 
for idx in (np.arange(0, len(toga_segments))):
    # find segment, get info 
    toga_ut = toga_segments.iloc[idx]
    idx_info = pd.Series({'Time_UTC': toga_ut['Time_UTC'], 'Flight': toga_ut['Flight']})
    # get ratio 
    toga_ut = toga_ut.drop('Time_UTC').drop('Intrument').drop('Flight')
    idx_rat = toga_ut/toga_bl
    # merge to info 
    to_append = idx_info.append(idx_rat)
    to_append = pd.DataFrame(to_append).transpose()
    # append
    toga_seg_ratios = toga_seg_ratios.append(to_append, ignore_index = True)


In [8]:
### add attribute to each species (BL, tropo, and UT lifetimes)
fill = pd.DataFrame([[np.nan, np.nan, np.nan, np.nan]], columns=['Trace_Gas', 'BL_tau', 'TROPO_tau', 'UT_tau'])
fill = fill.append(fill)

# --------- AWAS 
# read in from xls file to make dataframe - awas
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/awas/awas_lifetimes_12162019.xlsx'
awas_lifetimes = pd.read_excel(mypath2)
awas_lifetimes = fill.append(awas_lifetimes)
awas_lifetimes = awas_lifetimes.reset_index().drop('index', axis = 1)

# --------- TOGA
# read in from xls file to make dataframe - toga 
mypath2 = '/Volumes/scdrive2/TTS_2020/contrast_readin/toga_lodhalf/toga_lifetimes_12162019.xlsx'
toga_lifetimes = pd.read_excel(mypath2)
toga_lifetimes = fill.append(toga_lifetimes)
toga_lifetimes = toga_lifetimes.reset_index().drop('index', axis = 1)

In [9]:
# combine data from both instruments to one dataframe 
# --------- AWAS 
awas_seg_ratios = awas_seg_ratios.transpose()
awas_seg_ratios.insert(0, 'Instrument', 'AWAS')

# add lifetimes 
awas_seg_ratios.insert(1, 'BL_tau', awas_lifetimes['BL_tau'].values)
awas_seg_ratios.insert(2, 'TROPO_tau', awas_lifetimes['TROPO_tau'].values)
awas_seg_ratios.insert(3, 'UT_tau', awas_lifetimes['UT_tau'].values)

# --------- TOGA 
toga_seg_ratios = toga_seg_ratios.transpose()
toga_seg_ratios.insert(0, 'Instrument', 'TOGA')

# add lifetimes 
toga_seg_ratios.insert(1, 'BL_tau', toga_lifetimes['BL_tau'].values)
toga_seg_ratios.insert(2, 'TROPO_tau', toga_lifetimes['TROPO_tau'].values)
toga_seg_ratios.insert(3, 'UT_tau', toga_lifetimes['UT_tau'].values)

In [10]:
# combine data from both instruments to one dataframe 
toga_seg_ratios = toga_seg_ratios.drop('Time_UTC', axis = 0)
toga_seg_ratios = toga_seg_ratios.drop('Flight', axis = 0)

ratios_comb = awas_seg_ratios.append(toga_seg_ratios)
ratios_comb

,Instrument,BL_tau,TROPO_tau,UT_tau,0,1,2,3,4,5,...,164,165,166,167,168,169,170,171,172,173
Time_UTC,AWAS,NaN,NaN,NaN,2014-01-22 01:44:00,2014-01-22 01:55:15,2014-01-22 02:06:30,2014-01-22 02:17:45,2014-01-22 02:29:00,2014-01-22 02:37:56,...,2014-02-19 22:26:57,2014-02-22 00:37:11,2014-02-22 00:47:12,2014-02-22 00:57:11,2014-02-22 01:07:12,2014-02-22 01:15:43,2014-02-22 01:24:44,2014-02-22 01:33:43,2014-02-22 01:42:44,2014-02-22 01:51:44
Flight,AWAS,NaN,NaN,NaN,RF05,RF05,RF05,RF05,RF05,RF05,...,RF13,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14,RF14
C2H6_Ethane,AWAS,18.555721,48.949092,167.260709,0.50704,0.511892,0.436685,0.417277,0.674436,0.480353,...,0.641684,0.525235,0.561625,0.598016,0.606507,0.600442,0.584673,0.58346,0.557986,0.554347
C2H2_Ethyne,AWAS,5.422335,9.603369,18.712675,0.312471,0.345363,0.27958,0.27958,0.61672,0.320694,...,0.47693,0.353586,0.378255,0.378255,0.394701,0.386478,0.411147,0.41937,0.386478,0.386478
C3H8_Propane,AWAS,4.214557,9.062693,22.450000,0.149842,0.170275,0.102165,0.102165,0.599368,0.190708,...,0.326928,0.143031,0.163464,0.211141,0.183897,0.211141,0.20433,0.197519,0.156653,0.177086
C4H10_Isobutane,AWAS,2.175570,4.031311,7.903592,0.071025,0.0903955,0.0581114,0.0581114,0.665053,0.135593,...,0.193705,0.0839387,0.0516546,0.0968523,0.071025,0.103309,0.0839387,0.103309,0.0581114,0.103309
C4H10_nButane,AWAS,1.959376,3.813463,8.057685,0.0604664,0.0886841,0.100777,0.0765908,0.657069,0.149151,...,0.18543,0.0886841,0.0765908,0.0765908,0.0967463,0.084653,0.0644975,0.084653,0.084653,0.0644975
C5H12_nPentane,AWAS,1.214449,2.312606,4.723079,NaN,NaN,0.0469667,NaN,0.493151,0.0234834,...,0.234834,NaN,NaN,NaN,0.0234834,0.0469667,NaN,NaN,NaN,NaN
C6H6_Benzene,AWAS,3.755136,6.447323,11.947207,0.169869,0.180948,0.132941,0.129248,0.579772,0.192026,...,0.324968,0.265883,0.288039,0.243726,0.26219,0.26219,0.240033,0.284347,0.254804,0.258497
OCS_Carbonyl_sulfide,AWAS,2000.000000,2000.000000,2000.000000,1.00543,0.990776,1.00543,1.02557,1.0274,1.01642,...,0.972463,1.0274,0.992608,1.0036,1.00543,0.998102,1.00176,0.998102,0.9688,0.939498


In [11]:
# pickle this!! 
ratios_comb.to_pickle("./contrast_ratios_ut_seg.pkl")

In [12]:
##############################################################################################
##############################################################################################

In [18]:
# save tracer names 
# sort 
ratios_sort = ratios_comb
ratios_sort = ratios_sort.drop('Time_UTC').drop('Flight')
ratios_sort = ratios_sort.sort_values(['BL_tau'])

# save ordered bl tau as a variable 
tau = ratios_sort['BL_tau']

# drop uncecessary columns 
ratios_sort = ratios_sort.drop(columns = ['Instrument', 'BL_tau', 'TROPO_tau', 'UT_tau'])

In [19]:
# make a thing to fill with tracer names 
trcnames_allseg = pd.DataFrame()
for col in ratios_sort.columns: 
    trcnames_allseg[col] = ''
trcnames_allseg

,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173


In [39]:
# loop through each segment 
col = list(ratios_sort)
for i in col:
    utbl_full = np.array(ratios_sort[i].values, dtype=np.float64)
    utbl_not_null_idx = np.argwhere(~np.isnan(utbl_full))
    # ----- inputs without nans 
    my_utbl = utbl_full[utbl_not_null_idx]
    tracer_names = ratios_sort.index[utbl_not_null_idx]
    # ----- fill to make mu*, tau all length 52
    diffa = len(utbl_full) - len(my_utbl)
    if (diffa > 0):
        filla = np.empty((1,diffa))
        filla.fill(np.nan)
        my_utbl = np.append(my_utbl, filla)
        tracer_names = np.append(tracer_names, filla)
    else: 
        tracer_names = tracer_names.values 
        #tracer_names = np.ones([52])
        #tracer_names[:] = np.NaN
    # ----- save outputs of tracer names 
    trcnames_allseg[i] = tracer_names

In [ ]:
trcnames_allseg.to_pickle("./trcnames_allseg.pkl")